In [1]:
from copy import deepcopy


def read_file(path):
    res = open(path, "r").readlines()
    res = [list(x.strip()) for x in res]
    return res


def find_cell(grid, value):
    for x, l in enumerate(grid):
        for y, c in enumerate(l):
            if c == value:
                return (x, y)


def replace_values(grid, values, replacement):
    for x, l in enumerate(grid):
        for y, c in enumerate(l):
            if c in values:
                grid[x][y] = replacement
    return grid


def draw_grid(grid, path: list[(int, int, int)] | None = None):
    if path:
        path = [(x, y) for x, y, _ in path]

    grid = deepcopy(grid)
    print("\n\n")
    print("    " + "".join([f"{str(x):<2}" for x in range(len(grid[0]))]))

    if path:
        for x, y in path:
            grid[x][y] = "+"

    for i, l in enumerate(grid):
        l = f"{str(i):<2}" + "  " + " ".join(l)
        print(l)

In [ ]:
DIRECTIONS = [(0, 1), (1, 0), (0, -1), (-1, 0)]

grid = read_file("example")
START = find_cell(grid, "S")
END = find_cell(grid, "E")
grid = replace_values(grid, ["S", "E"], ".")
path = [START]


def in_grid(grid, x, y):
    return 0 <= x < len(grid) and 0 <= y < len(grid[0])


while path[-1] != END:
    x, y = path[-1]
    for dx, dy in DIRECTIONS:
        nx, ny = x + dx, y + dy
        if in_grid(grid, nx, ny) and grid[nx][ny] == "." and (nx, ny) not in path:
            path.append((nx, ny))

distances = {}
for i, p in enumerate(path):
    distances[p] = len(path) - i

cheats = {}

for t, coord in enumerate(path):
    x, y = coord
    for dx1, dy1 in DIRECTIONS:
        for dx2, dy2 in DIRECTIONS:
            nx, ny = x + dx1 + dx2, y + dy1 + dy2
            if not in_grid(grid, nx, ny) or grid[nx][ny] == "#":
                continue

            old = distances.get((nx, ny), 0)
            saved = len(path) - (t + old + 2)
            if len(path) - (t + old + 2) > 0:
                cheats[(x, y, nx, ny)] = saved

In [ ]:
from collections import namedtuple, Counter
from tqdm import tqdm

DIRECTIONS = [(0, 1), (1, 0), (0, -1), (-1, 0)]

grid = read_file("example")
START = find_cell(grid, "S")
END = find_cell(grid, "E")
grid = replace_values(grid, ["S", "E"], ".")
path = [START]


def in_grid(grid, x, y):
    return 0 <= x < len(grid) and 0 <= y < len(grid[0])


while path[-1] != END:
    x, y = path[-1]
    for dx, dy in DIRECTIONS:
        nx, ny = x + dx, y + dy
        if in_grid(grid, nx, ny) and grid[nx][ny] == "." and (nx, ny) not in path:
            path.append((nx, ny))

Cheat = namedtuple("Cheat", ["start", "end", "cheat_len", "saving"])


def cheat_on_path(path, p, cheat_len):
    res = []
    path2 = [(*x, i) for i, x in enumerate(path)]
    path2 = path2[path.index(p) :]
    x, y, d = path2[0]
    for i, j, d2 in path2:
        dx, dy = i - x, j - y
        distance = abs(dx) + abs(dy)
        saved = d2 - d - cheat_len
        if distance <= cheat_len and saved > 0:
            loop_res = Cheat((x, y), (i, j), distance, saved)
            res.append(loop_res)
    return res


cheats = []
for p in tqdm(path):
    r = cheat_on_path(path, p, 20)
    cheats += r

sorted(Counter([x.saving for x in cheats]).items(), key=lambda x: x[0])
# len([x for x in cheats if x.saving >= 50])

In [14]:
from collections import namedtuple, Counter
from tqdm import tqdm

DIRECTIONS = [(0, 1), (1, 0), (0, -1), (-1, 0)]

grid = read_file("input")
START = find_cell(grid, "S")
END = find_cell(grid, "E")
grid = replace_values(grid, ["S", "E"], ".")
path = [START]


def in_grid(grid, x, y):
    return 0 <= x < len(grid) and 0 <= y < len(grid[0])


while path[-1] != END:
    x, y = path[-1]
    for dx, dy in DIRECTIONS:
        nx, ny = x + dx, y + dy
        if in_grid(grid, nx, ny) and grid[nx][ny] == "." and (nx, ny) not in path:
            path.append((nx, ny))

Cheat = namedtuple("Cheat", ["start", "end", "cheat_len", "saving"])

distances = {}
for i, p in enumerate(path):
    distances[p] = len(path) - i

cheats = {}


max_len = 20
for t, p in tqdm(enumerate(path)):
    x, y = p
    for dx in range(x - max_len, x + max_len + 1):
        for dy in range(y - max_len, y + max_len + 1):
            distance = abs(dx - x) + abs(dy - y)
            if not in_grid(grid, dx, dy) or distance > max_len or grid[dx][dy] == "#":
                continue

            rem_t = distances[(dx, dy)]
            cheats[(x, y, dx, dy)] = len(path) - (t + rem_t + distance)

len([(k, v) for k, v in cheats.items() if v >= 100])

9357it [00:07, 1317.33it/s]


983905